In [1]:
import os
import warnings

import numpy as np
import pandas as pd
import scanpy as sc
from scipy import sparse
from tqdm.notebook import tqdm

from chemCPA.helper import rank_genes_groups_by_cov
from chemCPA.paths import DATA_DIR

sc.set_figure_params(dpi=100, frameon=False)
sc.logging.print_header()
warnings.filterwarnings('ignore')   

/nfs/staff-hdd/hetzell/miniconda3/envs/chemical_CPA/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
2023-08-15 23:35:33.556679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 23:35:36.190510: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-15 23:35:36.190662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.6


In [2]:
full = True 
load_adata = True
adata_in = 'lincs_full.h5ad' if full else 'lincs.h5ad'
adata = sc.read(DATA_DIR/ adata_in) if load_adata else None

adata_out = ''.join(adata_in.split('.')[:-1]) + '_pp.h5ad'
adata_out

'lincs_full_pp.h5ad'

In [3]:
adata.obs['condition'] = adata.obs['pert_iname']
adata.obs['condition'] = adata.obs['condition'].str.replace('/','|')

adata.obs['cell_type'] = adata.obs['cell_id']
adata.obs['dose_val'] = adata.obs['pert_dose'].astype(float) / np.max(adata.obs['pert_dose'].astype(float))
adata.obs['cov_drug_dose_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str) + '_' + adata.obs.dose_val.astype(str)
adata.obs['cov_drug_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str)
adata.obs['eval_category'] = adata.obs['cov_drug_dose_name']
adata.obs['control'] = (adata.obs['condition'] == 'DMSO').astype(int)

# adata.obs['cov_drug_dose_name'] = adata.obs['cov_drug_dose_name'].str.replace('/','|')

In [4]:
pd.crosstab(adata.obs.condition, adata.obs.cell_type)

cell_type,A375,A549,A673,AGS,ASC,ASC.C,BT20,CD34,CL34,CORL23,...,SW620,SW948,T3M10,THP1,TYKNU,U266,U937,VCAP,WSUDLCL2,YAPC
condition,,,,,,,,,,,,,,,,,,,,,
(+)-3-(1-propyl-piperidin-3-yl)-phenol,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"(+|-)-7-hydroxy-2-(N,N-di-n-propylamino)tetralin",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"1,2,3,4,5,6-hexabromocyclohexane",5,5,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"1,2,3,4-tetrahydroisoquinoline",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"1,2-dichlorobenzene",3,6,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zonisamide,23,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,0,18
zopiclone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zosuquidar,20,6,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,10,0,17


In [5]:
drug_abundance = adata.obs.condition.value_counts()
suff_drug_abundance = drug_abundance.index[drug_abundance>5]

In [6]:
# Delete conditions isufficient # of observations
adata = adata[adata.obs.condition.isin(suff_drug_abundance)].copy()
adata 

AnnData object with n_obs × n_vars = 1023036 × 978
    obs: 'cell_id', 'det_plate', 'det_well', 'lincs_phase', 'pert_dose', 'pert_dose_unit', 'pert_id', 'pert_iname', 'pert_mfc_id', 'pert_time', 'pert_time_unit', 'pert_type', 'rna_plate', 'rna_well', 'condition', 'cell_type', 'dose_val', 'cov_drug_dose_name', 'cov_drug_name', 'eval_category', 'control'
    var: 'pr_gene_title', 'pr_is_lm', 'pr_is_bing'
    uns: 'cydata_pull'

Calculate differential genes manually, such that the genes are the same per condition.

In [7]:
%%time

de_genes = {}
de_genes_quick = {}

adata_df = adata.to_df()
adata_df['condition'] = adata.obs.condition
dmso = adata_df[adata_df.condition == "DMSO"].mean()

for cond, df in tqdm(adata_df.groupby('condition')): 
    if cond != 'DMSO':
        drug_mean = df.mean()
        de_50_idx = np.argsort(abs(drug_mean-dmso))[-50:]
        de_genes_quick[cond] = drug_mean.index[de_50_idx].values

if full: 
    de_genes = de_genes_quick

else:
    sc.tl.rank_genes_groups(
        adata,
        groupby='condition', 
        reference='DMSO',
        rankby_abs=True,
        n_genes=50
    )
    for cond in tqdm(np.unique(adata.obs['condition'])):
        if cond != 'DMSO':
            df = sc.get.rank_genes_groups_df(adata, group=cond)  # this takes a while
            de_genes[cond] = df['names'][:50].values


  0%|          | 0/17990 [00:00<?, ?it/s]

CPU times: user 53.7 s, sys: 3.43 s, total: 57.2 s
Wall time: 56.9 s


Mapping from `rank_genes_groups_cov` might cause problems when drug contains '_'

In [8]:
def extract_drug(cond): 
    split = cond.split('_')
    if len(split) == 2: 
        return split[-1]
    return '_'.join(split[1:-1])

adata.obs['cov_drug_dose_name'].apply(lambda s: len(s.split('_'))).value_counts()
adata.obs['eval_category'].apply(lambda s: len(s.split('_'))).value_counts()

3    1022382
4        654
Name: eval_category, dtype: int64

In [9]:
adata.uns['rank_genes_groups_cov'] = {cat: de_genes_quick[extract_drug(cat)] for cat in adata.obs.eval_category.unique() if extract_drug(cat) != 'DMSO'}

In [10]:
adata.uns['rank_genes_groups_cov']

{'A375_aminoguanidine_0.05630630437109413': array(['NARFL', 'DYNLT3', 'KIAA0100', 'NPEPL1', 'HN1L', 'FUT1', 'ARNT2',
        'CDCA4', 'APOE', 'TEX10', 'POLR2I', 'AURKB', 'SCARB1', 'TESK1',
        'CSK', 'CD58', 'USP22', 'LRRC41', 'CHIC2', 'NR2F6', 'MMP2',
        'NPDC1', 'NOSIP', 'ECH1', 'PCNA', 'RTN2', 'CCND1', 'PAPD7',
        'MRPL12', 'BNIP3L', 'MYBL2', 'AKT1', 'METRN', 'ZNF586', 'SNCA',
        'CCNB1', 'S100A4', 'HSPA8', 'ACD', 'PAF1', 'CIRBP', 'SCAND1',
        'CHAC1', 'KIF2C', 'COG7', 'SUPV3L1', 'VPS28', 'ZNF274', 'CLTB',
        'GAPDH'], dtype=object),
 'A375_aminoguanidine_0.01876874935492992': array(['NARFL', 'DYNLT3', 'KIAA0100', 'NPEPL1', 'HN1L', 'FUT1', 'ARNT2',
        'CDCA4', 'APOE', 'TEX10', 'POLR2I', 'AURKB', 'SCARB1', 'TESK1',
        'CSK', 'CD58', 'USP22', 'LRRC41', 'CHIC2', 'NR2F6', 'MMP2',
        'NPDC1', 'NOSIP', 'ECH1', 'PCNA', 'RTN2', 'CCND1', 'PAPD7',
        'MRPL12', 'BNIP3L', 'MYBL2', 'AKT1', 'METRN', 'ZNF586', 'SNCA',
        'CCNB1', 'S100A4', 'HSP

In [11]:
adata.obs['split'] = 'train'

# take ood from top occurring perturbations to avoid losing data on low occ ones
ood_idx = sc.pp.subsample(
    adata[adata.obs.condition.isin(list(adata.obs.condition.value_counts().index[1:50]))],
    .1,
    copy=True
).obs.index
adata.obs['split'].loc[ood_idx] = 'ood'

# take test from a random subsampling of the rest
test_idx = sc.pp.subsample(
    adata[adata.obs.split != 'ood'],
    .16,
    copy=True
).obs.index
adata.obs['split'].loc[test_idx] = 'test'

In [12]:
pd.crosstab(adata.obs['split'], adata.obs['condition'])

condition,(+)-3-(1-propyl-piperidin-3-yl)-phenol,"(+|-)-7-hydroxy-2-(N,N-di-n-propylamino)tetralin","1,2,3,4,5,6-hexabromocyclohexane","1,2,3,4-tetrahydroisoquinoline","1,2-dichlorobenzene","1,2-propylene-glycol",1-benzylimidazole,1-methylisoquinoline,1-monopalmitin,1-phenylbiguanide,...,ziprasidone,zofenopril-calcium,zolantidine,zolmitriptan,zolpidem,zonisamide,zopiclone,zosuquidar,zoxazolamine,zuclopenthixol
split,,,,,,,,,,,,,,,,,,,,,
ood,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test,6,2,8,3,12,5,10,5,7,9,...,34,24,9,21,37,28,5,32,15,13
train,12,16,29,15,59,22,39,13,27,19,...,155,102,55,91,168,195,7,150,77,70


In [13]:
try: 
    del(adata.uns['rank_genes_groups'])  # too large
except: 
    print('All good.')

All good.


In [14]:
# code compatibility
adata.X = sparse.csr_matrix(adata.X)

In [15]:
sc.write(DATA_DIR / adata_out, adata)

In [16]:
print('all done.')

all done.


### Check that `adata.uns[rank_genes_groups_cov]` has all entries in `adata.obs.cov_drug_name` as keys

In [17]:
for i, k in enumerate(adata.obs.eval_category.unique()):
    try: 
        adata.uns['rank_genes_groups_cov'][k]
    except: 
        print(f"{i}: {k}") if 'DMSO' not in k else None

### Checking the same for the stored adata object

In [18]:
adata_2 = sc.read(DATA_DIR/ adata_out)

In [19]:
for i, k in enumerate(adata_2.obs.eval_category.unique()):
    try: 
        adata_2.uns['rank_genes_groups_cov'][k]
    except: 
        print(f"{i}: {k}") if 'DMSO' not in k else None

In [20]:
set(list(adata.uns['rank_genes_groups_cov'])) - set((list(adata_2.uns['rank_genes_groups_cov'])))

set()

In [21]:
set((list(adata_2.uns['rank_genes_groups_cov']))) - set(list(adata.uns['rank_genes_groups_cov']))

set()

In [22]:
len(list(adata_2.uns["rank_genes_groups_cov"].keys()))

218759

In [24]:
adata.obs["dose_val"].value_counts()

 0.056306    441932
-3.750000     43907
 0.028153     36233
 0.006256     32694
 0.018769     32677
              ...  
 0.690034         2
 0.001858         1
 0.006306         1
 0.056811         1
 0.056512         1
Name: dose_val, Length: 2934, dtype: int64